In [ ]:
!pip install numpy
!pip install pandas
!pip install scikit-learn
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Create New Dataset (1)
# Make a dataframe with drug name as rows, medical condition as columns
# and average rating score of drug for the medical condition as elements
dataset = pd.read_csv("../data/drugs_side_effects_drugs_com.csv")

# dataset.info()
cols = ["drug_name", "medical_condition", "rating"]
dataset_cleaned = dataset[cols].copy()
type(dataset_cleaned)
dataset_cleaned["rating"] = pd.to_numeric(dataset_cleaned["rating"], errors="coerce")
dataset_cleaned = dataset_cleaned.dropna(subset=['rating'])
dataset_cleaned.info()

dataset_rows = dataset_cleaned["drug_name"].unique() # Take row name
dataset_cols = dataset_cleaned["medical_condition"].unique() # Take column name
print(dataset_rows)
print(dataset_cols)

new_dataset = pd.DataFrame(0.0, index=dataset_rows, columns=dataset_cols) 
# Initialize new dataframe with elements as 0

<class 'pandas.core.frame.DataFrame'>
Index: 1586 entries, 0 to 2926
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   drug_name          1586 non-null   object 
 1   medical_condition  1586 non-null   object 
 2   rating             1586 non-null   float64
dtypes: float64(1), object(2)
memory usage: 49.6+ KB
['doxycycline' 'spironolactone' 'minocycline' ...
 'chorionic gonadotropin (hcg)' 'Lomaira' 'benzphetamine']
46


In [4]:
# Create New Dataset (2)

# Input the new dataframe with the values of drug for medical conditions in the row and column
def get_value(dname, mcond):
    for row in dataset_cleaned.itertuples():
        if (row.drug_name == dname and row.medical_condition == mcond):
            return row.rating
    return 0.0

for row in dataset_rows:
    for col in dataset_cols:
        new_dataset.at[row, col] = get_value(row, col)


new_dataset.describe()

# output = new_dataset 
# output.to_csv('newdataset.csv', index=True) # to check new form of dataset

,Acne,ADHD,AIDS/HIV,Allergies,Alzheimer's,Angina,Anxiety,Asthma,Bipolar Disorder,Bronchitis,...,Pain,Pneumonia,Psoriasis,Rheumatoid Arthritis,Schizophrenia,Seizures,Stroke,Swine Flu,UTI,Weight Loss
count,1573.000000,1573.000000,1573.000000,1573.000000,1573.000000,1573.000000,1573.000000,1573.000000,1573.000000,1573.000000,...,1573.000000,1573.000000,1573.000000,1573.000000,1573.000000,1573.000000,1573.000000,1573.000000,1573.000000,1573.000000
mean,0.571456,0.237953,0.291227,0.019962,0.048252,0.099555,0.218055,0.151685,0.186713,0.118627,...,0.447743,0.056135,0.185442,0.103115,0.053338,0.142276,0.015003,0.021996,0.087413,0.086586
std,2.055148,1.300016,1.488933,0.327836,0.578400,0.925766,1.313060,1.070845,1.133268,0.850903,...,1.834381,0.686304,1.183538,0.845560,0.630549,1.048044,0.349887,0.441068,0.835401,0.811129
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,10.000000,10.000000,10.000000,7.300000,10.000000,10.000000,10.000000,10.000000,10.000000,9.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,9.000000


In [5]:
# SVD Operation
def createSVD(A):
    # A is a np.array
    
    AtA = np.dot(A.T, A) 
    # dot: dot product, in this case, matrix multiplication; T = transposed array

    eigvals, eigvecs = np.linalg.eig(AtA) #Calculate eigen values and vectors
    ncols = np.argsort(eigvals) [::-1] 
    #sorts the eigenvalues in descending order and stores the corresponding indices in ncols

    Vt = eigvecs[:,ncols].T 
    # the same order as the sorted eigenvalues, dont forget to transpose
    
    sigma = np.sqrt(np.abs(eigvals[ncols]))
    
    U = np.dot(A, Vt.T) / (sigma + 10**(-10)) 
    # V is used rather than Vt, hence we retranspose Vt

    return U, sigma, Vt

#Truncated SVD Operation
def truncateSVD(A, k):
    U, sigma, Vt = createSVD(A)
    U_trunc = U[:, :k] # Take only the first k rows
    sigma_trunc = np.diag(sigma[:k]) # To make into diagonal matrix
    Vt_trunc = Vt[:k, :] # Take only the first k cols
    new_A = U_trunc @ sigma_trunc @ Vt_trunc 
    return new_A



In [6]:
# Make the truncated matrix
dataset_array = new_dataset.to_numpy()
print(type(new_dataset))
print(type(dataset_array))
dataset_array_trunc = truncateSVD(dataset_array, 50)

# Find Cosine Similarity
item_similarity = cosine_similarity(pd.DataFrame(dataset_array_trunc))

item_sim_df = pd.DataFrame(item_similarity, index=new_dataset.index, columns=new_dataset.index)
# item_sim_df.to_csv('sim50.csv', index=True) # To check the similarity matrix

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


In [9]:
def find_similar_drugs(drug_name, top_k=10):
    print("Finding drugs...")
    # Check if drug exists
    if drug_name not in item_sim_df.index:
        print(f"Error: Drug '{drug_name}' not found.")
        return None

    # Take similarity scores
    drug_scores_unfiltered = item_sim_df.loc[drug_name]
    drug_scores = drug_scores_unfiltered[drug_scores_unfiltered<0.99999]
    # If the similarity score is very high (near to 1), it means they are already used for similar purposes
    drug_scores = drug_scores[drug_scores_unfiltered > 0.1]
    # If the similarity score is very low (near to 0), it means they are completely different type of drugs

    # Sort
    sorted_drugs = drug_scores.sort_values(ascending=False).iloc[: top_k]

    # Show result
    if (sorted_drugs.empty):
        print(f"Alternative drugs for {drug_name} not found")
    else:
        print(f"Alternative drugs for {drug_name}")
    
    rank = 1
    for similar_drug, score in sorted_drugs.items():
        print(f"{rank}. {similar_drug} \t\t(Similarity: {score:.4f})")
        rank += 1


# To test 
drug_name = input()
find_similar_drugs(drug_name)
# Try "erythromycin", "caffeine", "amoxicillin", "aspirin"

Finding drugs...
Alternative drugs for Claritin
1. beclomethasone 		(Similarity: 0.7557)
2. budesonide 		(Similarity: 0.7375)
3. cromolyn 		(Similarity: 0.7311)
4. ciclesonide 		(Similarity: 0.6748)
5. mometasone 		(Similarity: 0.5417)
6. triamcinolone 		(Similarity: 0.4573)
